In [1]:
import sys
sys.path.append("C:/Users/E1005279/OneDrive - Sanlam Life Insurance Limited/MWL/MWLrepo/MiWayLife2/Data_Analysis_Package")
from data_preparation import DataPreparation, Analysis, multi_data_ops
import data_operations as do

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

import data_transforations as data_tr

# Loading data

In [2]:
path = "C:/Users/E1005279/OneDrive - Sanlam Life Insurance Limited/MWL/Lapse Rate Projections/Data/"

calls_data = pd.DataFrame(pd.read_csv(path+"call_reason.csv"))
sales_data = pd.DataFrame(pd.read_csv(path+"sales.csv"))
policy_data = pd.DataFrame(pd.read_csv(path+"policy_data.csv"))
payment_data = pd.concat([pd.DataFrame(pd.read_csv(path+"Payment_data_1.csv")),pd.DataFrame(pd.read_csv(path+"Payment_data_2.csv"))])
lapse_data = pd.concat([pd.DataFrame(pd.read_csv(path+"lapse_1.csv")),pd.DataFrame(pd.read_csv(path+"lapse_2.csv"))])


In [7]:
payment_data.columns

Index(['policy_id', 'collections_id', 'sales_channel', 'collection_date',
       'cover_start_date_original', 'cover_start_date',
       'cancellation_effective_date', '* policy start delay months',
       '* policy duration months', 'anniversary_due', 'amount',
       'collected_amount', 'collection_status', 'collection_sub_type',
       'payment_method', 'payment_type', 'payment_provider',
       'previous_collection_rate'],
      dtype='object')

# Filtering data to relevant policies

Relevant policies are policies that have been incepted. The rpt_datamodel_lapse has incepted policies only, hence filtering on policies within that dataset

In [3]:
pol_names = list(lapse_data['policy_name'])
sales_data = sales_data[sales_data['policy_name'].isin(pol_names)]
pol_ids= list(sales_data['policy_id'])
calls_data = calls_data[calls_data['policy_name'].isin(pol_names)]
payment_data = payment_data[payment_data['policy_id'].isin(pol_ids)]
policy_data = policy_data[policy_data['policy_name'].isin(pol_names)]

#Make date column of data type datetime
sales_data=do.dtype_datetime(sales_data)
calls_data=do.dtype_datetime(calls_data)
payment_data=do.dtype_datetime(payment_data)
policy_data=do.dtype_datetime(policy_data)
lapse_data=do.dtype_datetime(lapse_data)


In [4]:
len(set(pol_names)),len(set(pol_ids)),sales_data['policy_id'].nunique(), calls_data['policy_name'].nunique(), payment_data['policy_id'].nunique(),policy_data['policy_name'].nunique()

(65364, 65364, 65364, 39590, 63981, 65364)

# Aggregating Calla data, Payments data and Lapse data

## Calls data

<ol> <li>Remove call records that are related to cancellation of policy</li>
<li>Categorise the resolution codes</li>
<li>Add the policy's sale date</li>
<li>Filter call records to calls made after the sale date</li>
<li>Aggregate the calls data to policy_name</li>
</ol>

In [8]:
calls_data

,policy_name,call_id,contact_indicator,datetime_start,resolution_code
2,ML22-0000003,817c8493-27db-4e6b-8f03-bcbfcd51aa28,0,2022-08-31 10:23:12,completed
3,ML22-0000003,e5af692f-1fcf-48ee-91c6-97ae7d8f9367,0,2022-08-31 10:24:18,completed
4,ML22-0000014,d631ead4-a76e-46a7-8060-83b084a82ddb,1,2022-09-02 07:46:01,sale
12,ML22-0000005,4d104dc3-ece8-4226-ac34-404d4af38a9f,0,2022-09-02 10:54:29,completed
13,ML22-0000005,2375400c-833e-4cc2-90b1-95baa946c9c7,0,2022-09-02 12:21:04,completed
...,...,...,...,...,...
948982,MWL008540683,6308801b-9229-422d-a4b5-4a73f2f37722,0,2025-08-06 14:55:04,NaN
948988,ML23-0118810,f591438c-c444-425a-80a5-a5d224858351,0,2025-08-06 14:56:34,NaN
948992,ML25-0389266,1a9f33ad-c152-4f35-bb83-86b7c36ea982,0,2025-08-06 14:59:12,NaN
948997,ML23-0056798,99d0c75a-a769-48d9-bd26-3381ed9c05e0,1,2025-08-06 15:06:34,completed


In [9]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# NOTE: calls data might introduct target leakage due to calls being made to policyholders when payments are not made or when cancellation is requested.
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

#Remove calls hade with the following resolution codes
# related to cancellation resolution codes ['retentions_policy_cancelled','policy_cancelled','unpaid_cancelled_policy']

calls = DataPreparation(df=calls_data)
calls.cleaning_ops(drop_nulls = 'N',filtering = 'Y',filter_conditions_exclude={'resolution_code':['retentions_policy_cancelled','policy_cancelled','unpaid_cancelled_policy']})
calls.df.shape
calls_data=calls.df

C:\Users/E1005279/OneDrive - Sanlam Life Insurance Limited/MWL/MWLrepo/MiWayLife2/Data_Analysis_Package\data_operations.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_datetime(df[col], errors='coerce')
C:\Users/E1005279/OneDrive - Sanlam Life Insurance Limited/MWL/MWLrepo/MiWayLife2/Data_Analysis_Package\data_preparation.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df.drop_duplicates(inplace=True)


(329630, 5)

In [22]:
calls_data_res_cd = pd.DataFrame(pd.read_csv(path+"call_reason.csv"))

In [23]:
calls_data_res_cd['resolution_code'].unique()

array(['web_quote_declined', 'completed', 'sale',
       'not_interested_does_not_need_cover', 'no_sale_uw_declined',
       'not_interested_no_reason', 'no_sale_price_affordability',
       'no_sale_unreachable_after_initial_contact',
       'web_quote_not_interested_after_follow_up',
       'no_sale_product_Benefits_not_enough', 'unreachable_hung_up',
       'unreachable_voicemail_only', 'unreachable_no_answer',
       'price_beat_case_created',
       'cannot_offer_cover_not_the_life_insured',
       'no_sale_product_funeral_only',
       'not_interested_quick_quote_was_too_much',
       'no_sale_price_not_competitive', 'sysytem_duplicate',
       'web_quote_expired', 'duplicate',
       'not_interested_competitors_took_offer_from_first_contact',
       'not_interested_competitors_already_covered',
       'web_quote_web_preferred', 'testing', 'unreachable_wrong_number',
       'not_interested_unreachable_after_initial_contact',
       'retentions_policy_cancelled',
       'not_inter

In [ ]:
# creating buckets for resolution code to hot-one encode it and not have too many columns

def res_code_categories_postsale(code):
    if code in ['debit_date_changed', 'cover_reduced','requested_changes_completed','banking_details_changed','alternative_payer','cover_increased','commencement_moved',
                'additional_lives_update','bmi_update','banking_details_update', 'beneficiary_update','occupation_update','personal_details_update',
                 'educational_level_update','income_brackets_update','alcohol_intake_update']:
        return "update"
    elif code in ['claim_no_docs','claim_no_claim','claim_repudiated', 'claim','claim_accepted']:
        return "claims"
    elif code in['qa_no_risk','qa_risk_absorbed',  'qa_recaptured']:
        return "QA"
    elif code in ['sent_to_retentions', 'free_cover','premium_holiday_added','unpaid_policy_reduced', 'second_debit_scheduled','once_off_payment', 'policy_retained', 'unpaid_collect_again',
                  'policy_reinstated', 'policy_disputed','double_debit_scheduled']:
        return "payment"
    elif code in ['completed', 'no_contact_post_sale','product_explanation', 'sum_assured_disclosed', 'confirm_acceptance','converted', ]:
        return "sale"
    elif code in ['duplicate', 'no_changes_requested','unreachable_no_answer', 'testing', 'auto_close', 'sale','sysytem_duplicate', 'cannot_offer_cover_not_the_life_insured', 'no_sale_web_preferred',
       'not_interested_no_reason','not_interested_unreachable_after_initial_contact','not_interested_does_not_need_cover', 'unreachable_bulk_close',
       'unconverted',  'unreachable_voicemail_only','web_quote_declined', 'no_sale_quote_expired',  'not_interested_in_a_will','not_interested_competitors_already_covered', 
       'reading_script','note_on_behalf_of_sales_agent','unreachable_hung_up',  'cannot_offer_cover_age_too_old','no_sale_unreachable_after_initial_contact', 'documents_posted',
       'web_quote_not_interested_after_follow_up', 'not_interested_irate_do_not_call', 'no_sale_price_affordability']:
        return "other"
    else:
        return np.nan

def res_code_categories(code):
    if code in ['debit_date_changed','cover_reduced','requested_changes_completed','banking_details_changed',
       'commencement_moved','policy_disputed','cover_increased','additional_lives_update', 'bmi_update', 'banking_details_update','personal_details_update', 'income_brackets_update',
       'beneficiary_update','occupation_update', 'educational_level_update','alcohol_intake_update', 'documents_posted']:
        return "update"
    elif code in ['claim_accepted','claim','claim_repudiated', 'claim_no_claim','claim_no_docs']:
        return "claims"
    elif code in['qa_risk_absorbed','qa_recaptured','qa_no_risk']:
        return "QA"
    elif code in ['policy_cancelled', 'once_off_payment','alternative_payer','policy_retained','unpaid_collect_again','unpaid_policy_reduced',
        'second_debit_scheduled','policy_reinstated','double_debit_scheduled','free_cover', 'premium_holiday_added','sent_to_retentions']:
        return "payment"
    elif code in ['sale', 'no_sale_product_funeral_only','no_contact_post_sale','converted', 'product_explanation','sum_assured_disclosed','confirm_acceptance']:
        return "sale"
    elif code in ['web_quote_web_preferred','quoted_before']:
        return "quote"
    elif code in ['web_quote_declined','not_interested_does_not_need_cover', 'no_sale_uw_declined','not_interested_no_reason', 'no_sale_price_affordability','no_sale_unreachable_after_initial_contact',
       'web_quote_not_interested_after_follow_up','no_sale_product_Benefits_not_enough', 'not_interested_quick_quote_was_too_much','no_sale_price_not_competitive','web_quote_expired','not_interested_competitors_took_offer_from_first_contact',
       'not_interested_competitors_already_covered','not_interested_unreachable_after_initial_contact', 'not_interested_brand_negative_old_mutual','not_interested_affiliate_interested_in_competition',
       'not_interested_competitors_specific_provider','cannot_offer_cover_income_not_enough','not_interested_affiliate_cant_recall', 'no_sale_quote_expired','not_interested_affiliate_didnt_ask','not_interested_web_preferred','not_interested_hiv_test_do_not_want_to_go',
       'not_interested_brand_dont_know','no_sale_uw_exclusions','no_sale_web_preferred','not_interested_irate_do_not_call','not_interested_affiliate_did_not_opt_in','no_sale_product_retrenchment',
       'cannot_offer_cover_age_too_old', 'no_sale_product_short-term_insurance','no_sale_product_retirement_annuity','not_interested_waiting_for_loan_approval', 'unconverted',
       'cannot_offer_cover_citizenship_not_sa','not_interested_in_a_will', 'do_not_call', 'not_interested_cannot_afford','cannot_offer_cover_language_-_cannot_speak_english',
       'cannot_offer_cover_pensioner','quoted_not_interested','cannot_offer_cover_not_the_life_insured','unemployed']:
        return "no interest"
    elif code in ['completed',  'unreachable_hung_up','unreachable_voicemail_only', 'unreachable_no_answer','price_beat_case_created','sysytem_duplicate','testing', 'unreachable_wrong_number','duplicate', 'no_sale_timing_start_later','uw_manual','no_changes_requested', 
       'auto_close_external', 'uw_rule_change','auto_close', 'unreachable_bulk_close','invalidate_review','inbound_queries','existing_client','reading_script','note_on_behalf_of_sales_agent','busy']:
        return "other"
    else:
        return np.nan  


    
calls_data['res_code']= calls_data['resolution_code'].apply(res_code_categories_postsale)
calls_data




C:\Users\E1005279\AppData\Local\Temp\ipykernel_28560\2076976377.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calls_data['res_code']= calls_data['resolution_code'].apply(res_code_categories)


,policy_name,call_id,contact_indicator,datetime_start,resolution_code,res_code
2,ML22-0000003,817c8493-27db-4e6b-8f03-bcbfcd51aa28,0,2022-08-31 10:23:12,completed,sale
3,ML22-0000003,e5af692f-1fcf-48ee-91c6-97ae7d8f9367,0,2022-08-31 10:24:18,completed,sale
4,ML22-0000014,d631ead4-a76e-46a7-8060-83b084a82ddb,1,2022-09-02 07:46:01,sale,other
12,ML22-0000005,4d104dc3-ece8-4226-ac34-404d4af38a9f,0,2022-09-02 10:54:29,completed,sale
13,ML22-0000005,2375400c-833e-4cc2-90b1-95baa946c9c7,0,2022-09-02 12:21:04,completed,sale
...,...,...,...,...,...,...
948982,MWL008540683,6308801b-9229-422d-a4b5-4a73f2f37722,0,2025-08-06 14:55:04,NaN,NaN
948988,ML23-0118810,f591438c-c444-425a-80a5-a5d224858351,0,2025-08-06 14:56:34,NaN,NaN
948992,ML25-0389266,1a9f33ad-c152-4f35-bb83-86b7c36ea982,0,2025-08-06 14:59:12,NaN,NaN
948997,ML23-0056798,99d0c75a-a769-48d9-bd26-3381ed9c05e0,1,2025-08-06 15:06:34,completed,sale


In [11]:
#Adding sales date to calls data to filter to calls made after sale
calls_sale_date = multi_data_ops(file_list=[],df1=calls_data, df2=policy_data)
calls_sale_date.col_intersect

{'df1 | df2': {'policy_name'}}

In [12]:
calls_sale_date.merged_df = calls_sale_date.merging(on='policy_name', df_left=calls_sale_date.df1, df_right=calls_sale_date.df2, how='left', col_right=['policy_name','sale_date'])
calls_sale_date.merged_df

,policy_name,call_id,contact_indicator,datetime_start,resolution_code,res_code,sale_date
0,ML22-0000003,817c8493-27db-4e6b-8f03-bcbfcd51aa28,0,2022-08-31 10:23:12,completed,sale,2022-08-31
1,ML22-0000003,e5af692f-1fcf-48ee-91c6-97ae7d8f9367,0,2022-08-31 10:24:18,completed,sale,2022-08-31
2,ML22-0000014,d631ead4-a76e-46a7-8060-83b084a82ddb,1,2022-09-02 07:46:01,sale,other,2022-09-06
3,ML22-0000005,4d104dc3-ece8-4226-ac34-404d4af38a9f,0,2022-09-02 10:54:29,completed,sale,2022-09-02
4,ML22-0000005,2375400c-833e-4cc2-90b1-95baa946c9c7,0,2022-09-02 12:21:04,completed,sale,2022-09-02
...,...,...,...,...,...,...,...
329625,MWL008540683,6308801b-9229-422d-a4b5-4a73f2f37722,0,2025-08-06 14:55:04,NaN,NaN,2022-08-17
329626,ML23-0118810,f591438c-c444-425a-80a5-a5d224858351,0,2025-08-06 14:56:34,NaN,NaN,2023-11-14
329627,ML25-0389266,1a9f33ad-c152-4f35-bb83-86b7c36ea982,0,2025-08-06 14:59:12,NaN,NaN,2025-05-29
329628,ML23-0056798,99d0c75a-a769-48d9-bd26-3381ed9c05e0,1,2025-08-06 15:06:34,completed,sale,2023-08-07


In [13]:
#Selecting only call records after sale date
calls_data = calls_sale_date.merged_df[calls_sale_date.merged_df['datetime_start'].dt.strftime('%Y-%m-%d')>calls_sale_date.merged_df['sale_date'].dt.strftime('%Y-%m-%d')]


In [14]:
calls_data

,policy_name,call_id,contact_indicator,datetime_start,resolution_code,res_code,sale_date
6,ML22-0000005,302c00a7-8968-491d-8572-b7ee66a733af,0,2022-09-05 08:29:38,completed,sale,2022-09-02
270,ML22-0000180,be3c6b14-e50e-4b95-b3ef-9f6fadb40949,0,2022-09-21 14:59:24,completed,sale,2022-09-16
274,ML22-0000244,c069d070-6a63-4f13-933c-fdd36e0a9cbf,0,2022-09-21 15:09:21,completed,sale,2022-09-20
341,ML22-0000098,e1deeacb-d26e-4e11-ab97-a4141ea24a19,0,2022-09-26 07:15:42,completed,sale,2022-09-09
345,ML22-0000098,2ce916e4-6407-4013-85c1-990026858291,1,2022-09-26 08:39:40,completed,sale,2022-09-09
...,...,...,...,...,...,...,...
329625,MWL008540683,6308801b-9229-422d-a4b5-4a73f2f37722,0,2025-08-06 14:55:04,NaN,NaN,2022-08-17
329626,ML23-0118810,f591438c-c444-425a-80a5-a5d224858351,0,2025-08-06 14:56:34,NaN,NaN,2023-11-14
329627,ML25-0389266,1a9f33ad-c152-4f35-bb83-86b7c36ea982,0,2025-08-06 14:59:12,NaN,NaN,2025-05-29
329628,ML23-0056798,99d0c75a-a769-48d9-bd26-3381ed9c05e0,1,2025-08-06 15:06:34,completed,sale,2023-08-07


In [15]:
calls = DataPreparation(df=calls_data)

In [16]:
calls.null_df

,nulls,percentage nulls
policy_name,0,0.000000
call_id,0,0.000000
contact_indicator,0,0.000000
datetime_start,0,0.000000
resolution_code,2337,0.943141
res_code,2337,0.943141
sale_date,0,0.000000


In [17]:
calls.uniq_df

,no. of unique entries
policy_name,34470
call_id,247789
contact_indicator,2
datetime_start,244813
resolution_code,69
res_code,6
sale_date,2870


In [18]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# NOTE: calls data might introduct target leakage due to calls being made to policyholders when payments are not made or when cancellation is requested.
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

agg_dict={'# calls': pd.NamedAgg(column='call_id', aggfunc=pd.Series.nunique),
          '# calls contacted': pd.NamedAgg(column='contact_indicator', aggfunc='sum')
          }

calls.aggregation(groupby_cols=['policy_name'],cat_cols=['res_code'] ,agg_dict_not_cat_cols=agg_dict)
calls_data = calls.agg_df
calls.agg_df

,policy_name,# calls,# calls contacted,res_code_QA_sum,res_code_claims_sum,res_code_other_sum,res_code_payment_sum,res_code_sale_sum,res_code_update_sum
0,ML22-0000001,2,1,0,0,0,0,2,0
1,ML22-0000005,13,1,0,0,9,0,4,0
2,ML22-0000014,2,1,0,0,0,0,2,0
3,ML22-0000016,12,0,0,0,0,2,10,0
4,ML22-0000031,24,17,0,0,0,4,20,0
...,...,...,...,...,...,...,...,...,...
34465,SD25-0419225,3,1,0,0,0,0,3,0
34466,SD25-0419607,1,0,0,0,0,0,1,0
34467,SD25-0423742,1,1,0,0,0,0,1,0
34468,SD25-0426418,2,1,0,0,0,0,1,0


## Payments data
<ol><li>Exclude collections for current month and making anniversary date of datetime data type</li>
<li>Split claims and monthly premiums</li>
<li>Aggregate claims data</li>
<li>Adding a paid flag column, paid?</li>
<li>Aggregating to policy_id and noting which of last n months premiums were paid.</li>
<li>Adding the duration to next anniversary</li>
<li>Aggregate to policy_id</li>
<li>Create payment rate column (no. paid/total payments due) and create no. of anniversaries column</li>
<li>Merge last months paid data and claims data to the aggrewgated payments data</li>
</ol>

In [30]:
payment_data

,policy_id,collections_id,sales_channel,collection_date,cover_start_date_original,cover_start_date,cancellation_effective_date,* policy start delay months,* policy duration months,anniversary_due,amount,collected_amount,collection_status,collection_sub_type,payment_method,payment_type,payment_provider,previous_collection_rate
0,1e3163bd-fdc3-318d-2a9f-6629f7e5daec,180d7f3f-0f72-4da0-964d-eaa545c1b45a,Agent,2024-08-27,2015-04-01,2015-04-01,NaT,0,124,2026-04-01 00:00:00,273.07,273.07,successful,Monthly,EFT,DebiCheck,Qlink,1.0
1,1e3163bd-fdc3-318d-2a9f-6629f7e5daec,268ae25c-2c77-4b83-a549-b5aced1fe352,Agent,2024-07-26,2015-04-01,2015-04-01,NaT,0,124,2026-04-01 00:00:00,273.07,273.07,successful,Monthly,EFT,DebiCheck,Qlink,1.0
2,1e3163bd-fdc3-318d-2a9f-6629f7e5daec,2ed02008-ba68-4bed-bc8e-fdef080c7bf2,Agent,2025-06-27,2015-04-01,2015-04-01,NaT,0,124,2026-04-01 00:00:00,294.92,294.92,successful,Monthly,EFT,DebiCheck,Qlink,1.0
3,1e3163bd-fdc3-318d-2a9f-6629f7e5daec,39d7a57b-a938-4afe-b9db-bc74167f436f,Agent,2024-10-25,2015-04-01,2015-04-01,NaT,0,124,2026-04-01 00:00:00,273.07,273.07,successful,Monthly,EFT,DebiCheck,Qlink,1.0
4,1e3163bd-fdc3-318d-2a9f-6629f7e5daec,4d8515c8-f6e0-4cdb-aad8-0c3baaa31333,Agent,2024-12-27,2015-04-01,2015-04-01,NaT,0,124,2026-04-01 00:00:00,273.07,273.07,successful,Monthly,EFT,DebiCheck,Qlink,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135596,f3941032-11e9-70a6-9453-682716baa954,dcc2f0e0-50b5-417c-8dcc-f64242b08051,Agent,2025-06-30,2025-06-01,2025-06-01,NaT,0,2,2026-06-01 00:00:00,351.43,351.43,successful,Monthly,DebiCheck,DebiCheck,Qlink Debicheck,1.0
135597,f3941032-11e9-70a6-9453-682716baa954,f2719079-c082-4c4d-ab1f-08a11e98cdfe,Agent,2025-07-31,2025-06-01,2025-06-01,NaT,0,2,2026-06-01 00:00:00,351.43,351.43,successful,Monthly,DebiCheck,DebiCheck,Qlink Debicheck,1.0
135598,f6270840-79cf-45dd-ba57-67f8ed6c7296,530d00d8-3ff2-4407-9d9a-0b2689d57061,Digital - Assisted,2025-06-26,2025-06-01,2025-06-01,NaT,0,2,2026-06-01 00:00:00,384.63,384.63,successful,Monthly,DebiCheck,DebiCheck,Qlink Debicheck,1.0
135599,f6270840-79cf-45dd-ba57-67f8ed6c7296,7e5deee2-b79c-4d1b-b9f9-2364fb37037a,Digital - Assisted,2025-05-26,2025-06-01,2025-06-01,NaT,0,2,2026-06-01 00:00:00,384.63,384.63,successful,Monthly,DebiCheck,DebiCheck,Qlink Debicheck,NaN


In [ ]:
#Exclude payments for current month
payment_data = payment_data[payment_data['collection_date']< pd.Timestamp('2025-08-01')]
#making anniversary date a datetime dtype
payment_data['anniversary_due'] = pd.to_datetime(payment_data['anniversary_due'], errors='coerce')

In [ ]:
claims_data = payment_data[payment_data['collection_sub_type']=='Claim']
payment_data = payment_data[payment_data['collection_sub_type']=='Monthly']

In [34]:
claims = DataPreparation(df=claims_data)
agg_dict = {'# claims': pd.NamedAgg(column='collections_id', aggfunc=pd.Series.nunique)}
claims.aggregation(groupby_cols=['policy_id'],cat_cols=[],agg_dict_not_cat_cols=agg_dict)
claims.agg_df

,policy_id,# claims
0,107c0513-5c34-9300-2ac3-6629ffb12bab,1
1,109f5129-16ca-89a7-1d26-65fb7dbce6b7,1
2,10b94d19-f6a2-a758-ecee-65fb935f5f54,2
3,112ecc3d-9ab4-4de8-8f73-65fb435e0f95,1
4,11408adb-e64a-93bd-71aa-65fb91571d7b,1
...,...,...
749,f383cdf7-9848-4573-da9d-65d8d67a6e7c,1
750,f409c8e6-f58b-14b7-a488-645e01853332,1
751,f62405ae-1eef-0e78-ce6b-65fb7e418d31,1
752,f75a0bad-4d18-cf62-be41-65fb7f1d2819,1


In [48]:
payment_data['paid?']= payment_data['collected_amount'].apply(lambda x: 'paid' if pd.notnull(x) else 'unpaid')

C:\Users\E1005279\AppData\Local\Temp\ipykernel_28560\2757173233.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  payment_data['paid?']= payment_data['collected_amount'].apply(lambda x: 'paid' if pd.notnull(x) else 'unpaid')


In [78]:

def last_n_payments(group):
    sortby='collection_date'
    n=4
    new_val = 'paid?'
    sorted_group = group.sort_values(sortby,ascending=False)
    top_n = sorted_group[new_val].head(n).tolist()
    top_n += ['N/A']*(n-len(top_n))
    new_cols=[]
    for m in range(0,n):
        new_cols.append(str(m+1)+' month ago payment')
    return pd.Series(top_n, index=new_cols)

last_month_payments_data = payment_data.groupby('policy_id', as_index=False).apply(last_n_payments)

C:\Users\E1005279\AppData\Local\Temp\ipykernel_28560\3109358826.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  last_month_payments_data = payment_data.groupby('policy_id', as_index=False).apply(last_n_payments)


In [79]:
last_month_payments_data.columns

Index(['policy_id', '1 month ago payment', '2 month ago payment',
       '3 month ago payment', '4 month ago payment'],
      dtype='object')

In [ ]:

payment_data['duration to anniversary (months)'] = payment_data.apply(
    lambda row: ((row['anniversary_due'] - row['cancellation_effective_date']) / np.timedelta64(1, 'D'))//30
    if pd.notnull(row['cancellation_effective_date'])
    else ((row['anniversary_due'] - pd.Timestamp('2025-08-01')) / np.timedelta64(1, 'D'))//30,
    axis=1
)


C:\Users\E1005279\AppData\Local\Temp\ipykernel_28560\1292687195.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  payment_data['duration to anniversary'] = payment_data.apply(


,policy_id,collections_id,sales_channel,collection_date,cover_start_date_original,cover_start_date,cancellation_effective_date,* policy start delay months,* policy duration months,anniversary_due,amount,collected_amount,collection_status,collection_sub_type,payment_method,payment_type,payment_provider,previous_collection_rate,paid?,duration to anniversary
0,1e3163bd-fdc3-318d-2a9f-6629f7e5daec,180d7f3f-0f72-4da0-964d-eaa545c1b45a,Agent,2024-08-27,2015-04-01,2015-04-01,NaT,0,124,2026-04-01,273.07,273.07,successful,Monthly,EFT,DebiCheck,Qlink,1.0,paid,8.0
1,1e3163bd-fdc3-318d-2a9f-6629f7e5daec,268ae25c-2c77-4b83-a549-b5aced1fe352,Agent,2024-07-26,2015-04-01,2015-04-01,NaT,0,124,2026-04-01,273.07,273.07,successful,Monthly,EFT,DebiCheck,Qlink,1.0,paid,8.0
2,1e3163bd-fdc3-318d-2a9f-6629f7e5daec,2ed02008-ba68-4bed-bc8e-fdef080c7bf2,Agent,2025-06-27,2015-04-01,2015-04-01,NaT,0,124,2026-04-01,294.92,294.92,successful,Monthly,EFT,DebiCheck,Qlink,1.0,paid,8.0
3,1e3163bd-fdc3-318d-2a9f-6629f7e5daec,39d7a57b-a938-4afe-b9db-bc74167f436f,Agent,2024-10-25,2015-04-01,2015-04-01,NaT,0,124,2026-04-01,273.07,273.07,successful,Monthly,EFT,DebiCheck,Qlink,1.0,paid,8.0
4,1e3163bd-fdc3-318d-2a9f-6629f7e5daec,4d8515c8-f6e0-4cdb-aad8-0c3baaa31333,Agent,2024-12-27,2015-04-01,2015-04-01,NaT,0,124,2026-04-01,273.07,273.07,successful,Monthly,EFT,DebiCheck,Qlink,1.0,paid,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135596,f3941032-11e9-70a6-9453-682716baa954,dcc2f0e0-50b5-417c-8dcc-f64242b08051,Agent,2025-06-30,2025-06-01,2025-06-01,NaT,0,2,2026-06-01,351.43,351.43,successful,Monthly,DebiCheck,DebiCheck,Qlink Debicheck,1.0,paid,10.0
135597,f3941032-11e9-70a6-9453-682716baa954,f2719079-c082-4c4d-ab1f-08a11e98cdfe,Agent,2025-07-31,2025-06-01,2025-06-01,NaT,0,2,2026-06-01,351.43,351.43,successful,Monthly,DebiCheck,DebiCheck,Qlink Debicheck,1.0,paid,10.0
135598,f6270840-79cf-45dd-ba57-67f8ed6c7296,530d00d8-3ff2-4407-9d9a-0b2689d57061,Digital - Assisted,2025-06-26,2025-06-01,2025-06-01,NaT,0,2,2026-06-01,384.63,384.63,successful,Monthly,DebiCheck,DebiCheck,Qlink Debicheck,1.0,paid,10.0
135599,f6270840-79cf-45dd-ba57-67f8ed6c7296,7e5deee2-b79c-4d1b-b9f9-2364fb37037a,Digital - Assisted,2025-05-26,2025-06-01,2025-06-01,NaT,0,2,2026-06-01,384.63,384.63,successful,Monthly,DebiCheck,DebiCheck,Qlink Debicheck,NaN,paid,10.0


In [68]:
payment_data.columns

Index(['policy_id', 'collections_id', 'sales_channel', 'collection_date',
       'cover_start_date_original', 'cover_start_date',
       'cancellation_effective_date', '* policy start delay months',
       '* policy duration months', 'anniversary_due', 'amount',
       'collected_amount', 'collection_status', 'collection_sub_type',
       'payment_method', 'payment_type', 'payment_provider',
       'previous_collection_rate', 'paid?', 'duration to anniversary',
       'policy duration (months)'],
      dtype='object')

In [71]:
payments = DataPreparation(df=payment_data)
agg_dict={'sales_channel': pd.NamedAgg(column='sales_channel', aggfunc='first'),
          '* policy start delay months': pd.NamedAgg(column='* policy start delay months', aggfunc='max'),
          '* policy duration months': pd.NamedAgg(column='* policy duration months', aggfunc='max'),
          'last premium amount due': pd.NamedAgg(column='amount', aggfunc='last'),
          'duration to anniversary': pd.NamedAgg(column='duration to anniversary', aggfunc='max')}
payments.aggregation(groupby_cols=['policy_id'],cat_cols=['paid?','payment_method'],agg_dict_not_cat_cols=agg_dict)
payments.agg_df

,policy_id,sales_channel,* policy start delay months,* policy duration months,last premium amount due,duration to anniversary,paid?_paid_sum,paid?_unpaid_sum,payment_method_DebiCheck_sum,payment_method_EFT_sum,payment_method_Pre Fund_sum
0,100003c9-2094-3107-e604-65d8e6b4ec03,Agent,0,45,624.06,3.0,17,0,6,11,0
1,10000cc8-8f08-5e2c-b6ad-661d48606710,Agent,0,14,249.27,10.0,15,0,12,3,0
2,100307d2-6857-7f50-833c-66b391c37879,Agent,0,11,288.91,1.0,11,4,11,4,0
3,10048ce0-253d-aeff-4f59-64e76756bddf,Agent,0,22,193.82,2.0,23,0,12,11,0
4,10061226-ed6a-b733-47f0-65fb970086cd,Agent,0,48,388.10,0.0,16,0,0,16,0
...,...,...,...,...,...,...,...,...,...,...,...
63976,ffaa032e-2a01-cf52-cc1a-65fb3147c393,Agent,0,73,602.75,-2.0,15,4,0,19,0
63977,ffb7087c-78f7-a485-5b7f-65d8d25da2c7,Agent,0,62,349.90,10.0,17,2,0,19,0
63978,ffd20919-303c-e88e-fb1f-65fba1578790,Agent,0,34,200.68,2.0,15,1,0,16,0
63979,ffe1059c-beea-354d-27fa-6629fe94a5cf,Agent,0,80,883.94,4.0,15,1,0,16,0


In [72]:
payments.agg_df['# anniversaries']=payments.agg_df['* policy duration months']//12
payments.agg_df['payment rate']=payments.agg_df['paid?_paid_sum']/(payments.agg_df['paid?_paid_sum']+payments.agg_df['paid?_unpaid_sum'])

In [73]:
payments.agg_df

,policy_id,sales_channel,* policy start delay months,* policy duration months,last premium amount due,duration to anniversary,paid?_paid_sum,paid?_unpaid_sum,payment_method_DebiCheck_sum,payment_method_EFT_sum,payment_method_Pre Fund_sum,# anniversaries,payment rate
0,100003c9-2094-3107-e604-65d8e6b4ec03,Agent,0,45,624.06,3.0,17,0,6,11,0,3,1.000000
1,10000cc8-8f08-5e2c-b6ad-661d48606710,Agent,0,14,249.27,10.0,15,0,12,3,0,1,1.000000
2,100307d2-6857-7f50-833c-66b391c37879,Agent,0,11,288.91,1.0,11,4,11,4,0,0,0.733333
3,10048ce0-253d-aeff-4f59-64e76756bddf,Agent,0,22,193.82,2.0,23,0,12,11,0,1,1.000000
4,10061226-ed6a-b733-47f0-65fb970086cd,Agent,0,48,388.10,0.0,16,0,0,16,0,4,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63976,ffaa032e-2a01-cf52-cc1a-65fb3147c393,Agent,0,73,602.75,-2.0,15,4,0,19,0,6,0.789474
63977,ffb7087c-78f7-a485-5b7f-65d8d25da2c7,Agent,0,62,349.90,10.0,17,2,0,19,0,5,0.894737
63978,ffd20919-303c-e88e-fb1f-65fba1578790,Agent,0,34,200.68,2.0,15,1,0,16,0,2,0.937500
63979,ffe1059c-beea-354d-27fa-6629fe94a5cf,Agent,0,80,883.94,4.0,15,1,0,16,0,6,0.937500


In [83]:
payments_data = multi_data_ops(file_list=[],df1=payments.agg_df,df2=last_month_payments_data,df3=claims.agg_df)
pay_last = payments_data.merging(on='policy_id',df_left=payments_data.df1, df_right=payments_data.df2, how='left',col_left=['policy_id', 'sales_channel', '* policy start delay months',
        '* policy duration months', 'last premium amount due',
        'duration to anniversary','payment_method_DebiCheck_sum', 'payment_method_EFT_sum',
        'payment_method_Pre Fund_sum', '# anniversaries', 'payment rate'])
payments_data.merged_df = payments_data.merging(on='policy_id',df_left=pay_last, df_right=payments_data.df3, how='left')
payments_data.merged_df['# claims'] = payments_data.merged_df['# claims'].apply(lambda x: x if pd.notnull(x) else 0)
payments_data.merged_df

,policy_id,sales_channel,* policy start delay months,* policy duration months,last premium amount due,duration to anniversary,payment_method_DebiCheck_sum,payment_method_EFT_sum,payment_method_Pre Fund_sum,# anniversaries,payment rate,1 month ago payment,2 month ago payment,3 month ago payment,4 month ago payment,# claims
0,100003c9-2094-3107-e604-65d8e6b4ec03,Agent,0,45,624.06,3.0,6,11,0,3,1.000000,paid,paid,paid,paid,0.0
1,10000cc8-8f08-5e2c-b6ad-661d48606710,Agent,0,14,249.27,10.0,12,3,0,1,1.000000,paid,paid,paid,paid,0.0
2,100307d2-6857-7f50-833c-66b391c37879,Agent,0,11,288.91,1.0,11,4,0,0,0.733333,unpaid,unpaid,paid,paid,0.0
3,10048ce0-253d-aeff-4f59-64e76756bddf,Agent,0,22,193.82,2.0,12,11,0,1,1.000000,paid,paid,paid,paid,0.0
4,10061226-ed6a-b733-47f0-65fb970086cd,Agent,0,48,388.10,0.0,0,16,0,4,1.000000,paid,paid,paid,paid,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63976,ffaa032e-2a01-cf52-cc1a-65fb3147c393,Agent,0,73,602.75,-2.0,0,19,0,6,0.789474,unpaid,unpaid,paid,paid,0.0
63977,ffb7087c-78f7-a485-5b7f-65d8d25da2c7,Agent,0,62,349.90,10.0,0,19,0,5,0.894737,paid,paid,unpaid,paid,0.0
63978,ffd20919-303c-e88e-fb1f-65fba1578790,Agent,0,34,200.68,2.0,0,16,0,2,0.937500,paid,paid,paid,paid,0.0
63979,ffe1059c-beea-354d-27fa-6629fe94a5cf,Agent,0,80,883.94,4.0,0,16,0,6,0.937500,paid,unpaid,paid,paid,0.0


## Lapse data


In [4]:
lapse_data.columns

Index(['policy_name', 'age', 'cover_amount_full', 'policy_status',
       'current_policy_status', 'education', 'eml', 'pml',
       'expected_lapse_rate', 'exposure_status', 'gender', 'hiv_group',
       'individual_income', 'lapse_type', 'num_unpaid_months',
       'occupation_class', 'partner_income', 'policy_sale_date',
       'smoker_status'],
      dtype='object')

In [12]:
lapse = DataPreparation(df=lapse_data)
agg_dict={'cover_amount_full': pd.NamedAgg(column='cover_amount_full', aggfunc='mean'),
          'current_policy_status': pd.NamedAgg(column='current_policy_status', aggfunc='last'),
          'education': pd.NamedAgg(column='education', aggfunc='last'),
          'eml': pd.NamedAgg(column='eml', aggfunc='mean'),
          'pml': pd.NamedAgg(column='pml', aggfunc='mean'),
          'expected_lapse_rate': pd.NamedAgg(column='expected_lapse_rate', aggfunc='mean'),
          'gender': pd.NamedAgg(column= 'gender', aggfunc='last'),
          'hiv_group': pd.NamedAgg(column='hiv_group', aggfunc='last'),
          'current individual_income': pd.NamedAgg(column='individual_income', aggfunc='last'),
          'orginal individual_income': pd.NamedAgg(column='individual_income', aggfunc='first'),
          'lapse_type': pd.NamedAgg(column='lapse_type', aggfunc=lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan),
          'occupation_class': pd.NamedAgg(column='occupation_class', aggfunc='last'),
          'partner_income': pd.NamedAgg(column='partner_income', aggfunc='last'),
          'smoker_status': pd.NamedAgg(column='smoker_status', aggfunc='last')
          }
lapse.aggregation(groupby_cols=['policy_name'],cat_cols=[],agg_dict_not_cat_cols=agg_dict)
lapse.agg_df

,policy_name,cover_amount_full,current_policy_status,education,eml,pml,expected_lapse_rate,gender,hiv_group,current individual_income,orginal individual_income,lapse_type,occupation_class,partner_income,smoker_status
0,ML22-0000001,1.000000e+05,Cancelled,Matric + 4 year Diploma,-0.050,0.0,0.025799,Male,red,20000.0,20000.0,NaN,A,0,Non Smoker
1,ML22-0000003,1.136972e+06,Cancelled,Matric + certificate,-0.050,0.0,0.028797,Female,red,10000.0,10000.0,Client Selected,C,50000,Non Smoker
2,ML22-0000005,1.000000e+05,Cancelled,Matric + certificate,0.350,0.0,0.040727,Female,red,25000.0,25000.0,Client Selected,A,0,Non Smoker
3,ML22-0000014,1.000000e+06,Cancelled,Matric + 3 year Diploma,0.000,0.0,0.084850,Female,red,40000.0,40000.0,Client Selected,A,0,Smoker
4,ML22-0000016,1.272500e+06,Cancelled,Matric + 3 year Diploma,0.000,0.0,0.047571,Male,red,30000.0,30000.0,Payment Lapsed,A,50000,Smoker
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65359,SD25-0423742,5.000000e+05,Active,Matric,-0.050,0.0,NaN,Male,red,20000.0,20000.0,NaN,A,0,Non Smoker
65360,SD25-0425635,2.000000e+06,Active,Postgraduate Degree,-0.050,0.0,NaN,Female,red,63000.0,63000.0,NaN,A,0,Non Smoker
65361,SD25-0426418,5.250000e+05,Cancellation Requested,No Matric,0.175,0.0,NaN,Female,red,16000.0,16000.0,NaN,A,0,Non Smoker
65362,SD25-0426488,5.000000e+05,Active,No Matric,-0.050,0.0,NaN,Female,red,34000.0,34000.0,NaN,C,0,Non Smoker


In [97]:
lapse.agg_df['policy_name'].nunique()

65364

# Previous aggregtaions in functions

In [4]:
pre_inception_calls_data = data_tr.pre_sale_calls(calls_data,policy_data)
post_sale_calls_data = data_tr.post_sale_calls(calls_data,policy_data)
payment_hist_data = data_tr.payments_history(payment_data,current_date='2025-08-01')
lapse_data = data_tr.lapses(lapse_data)
sales_data = sales_data.drop_duplicates(subset=['policy_id'], keep='last')

c:\Users\E1005279\OneDrive - Sanlam Life Insurance Limited\MWL\MWLrepo\LapseRate_modeling\data_transforations.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calls.df['res_code']= calls.df['resolution_code'].apply(res_code_categories)
c:\Users\E1005279\OneDrive - Sanlam Life Insurance Limited\MWL\MWLrepo\LapseRate_modeling\data_transforations.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calls.df['res_code']= calls.df['resolution_code'].apply(res_code_categories_postsale)
c:\Users\E1005279\OneD

# Merging all the data

In [ ]:
sales_data = sales_data.drop_duplicates(subset=['policy_id'], keep='last')
lapse_data

## at point of inception (obj1)

In [15]:
at_inception_data = multi_data_ops(file_list=[],df1=pre_inception_calls_data, df2=sales_data,df3=policy_data,df4=lapse_data)
at_inception_data.col_intersect

{'df1 | df2': {'policy_name'},
 'df1 | df3': {'policy_name'},
 'df1 | df4': {'policy_name'},
 'df2 | df3': {'campaign_name', 'policy_id', 'policy_name'},
 'df2 | df4': {'education', 'gender', 'policy_name', 'smoker_status'},
 'df3 | df4': {'policy_name'}}

In [16]:
at_inception_data.df1.shape, at_inception_data.df2.shape, at_inception_data.df3.shape, at_inception_data.df4.shape

((19605, 11), (65364, 19), (65364, 47), (65364, 15))

In [17]:
at_inception_data.columns

{'df1': Index(['policy_name', '# calls', '# calls contacted', 'res_code_QA_sum',
        'res_code_claims_sum', 'res_code_no interest_sum', 'res_code_other_sum',
        'res_code_payment_sum', 'res_code_quote_sum', 'res_code_sale_sum',
        'res_code_update_sum'],
       dtype='object'),
 'df2': Index(['policy_id', 'policy_name', 'age', 'benefits_count', 'calls',
        'call_contacts', 'call_contact_rate', 'campaign_name', 'education',
        'gender', 'income', 'lead_provider_name', 'lead_type', 'occupation',
        'payment_profile', 'sales_channel', 'smoker_status',
        'sold_socio_economic_class', 'underwriting_outcome'],
       dtype='object'),
 'df3': Index(['Unnamed: 0.1', 'Unnamed: 0', 'securitygroup_id', 'policy_name',
        'sales_process_name', 'campaign_code', 'campaign_name', 'optionality',
        'hiv_test_required', 'policy_type', 'primary_contact_id',
        'policy_status', 'premium', 'cover_start_date_original',
        'cover_start_date', 'debit_order

In [ ]:
sale_policy = at_inception_data.merging(on='policy_name',df_left=at_inception_data.df2, df_right=at_inception_data.df3, how='inner', col_left=['policy_id', 'policy_name', 'age', 'benefits_count', 
        'education','gender', 'income', 'lead_provider_name', 'lead_type', 'occupation',
        'payment_profile', 'sales_channel', 'smoker_status',
        'sold_socio_economic_class', 'underwriting_outcome'], col_right=['securitygroup_id', 'policy_name', 'campaign_name', 'optionality',
        'hiv_test_required', 'policy_type', 'policy_status', 'premium', 'cover_start_date_original','cover_start_date', 
        'cancellation_effective_date','sale_date', 'original_premium',
        'cancellation_reason', 'anniversary_due', 'last_benefit_amount',
        'first_month_at_risk', 'last_month_at_risk', 'last_settled', 'SEC',
        'pricing_version', 'first_collected_date',
        'number_of_collection_attempts', 'number_of_successful_collections',
        'total_collected',  'net_collected',
        'last_main_premium', 'last_benefit_type', 'total_funeral_premium',
        'funeral_count', 'has_been_recaptured', 'reason', 'last_benefit_status', 'prev_premium', 'fixed_debit_day',
        'lapse_flag'])

In [20]:
sale_policy.shape

(65364, 49)

In [19]:
sale_policy_lapse = at_inception_data.merging(on='policy_name',df_left=sale_policy, df_right=at_inception_data.df4, how='inner', col_right=['policy_name', 'cover_amount_full', 'current_policy_status',
         'eml', 'pml', 'expected_lapse_rate', 'hiv_group',
        'current individual_income', 'orginal individual_income', 'lapse_type',
        'occupation_class', 'partner_income'])

In [21]:
sale_policy_lapse.shape

(65364, 60)

In [22]:
at_inception_data.df_merged = at_inception_data.merging(on='policy_name',df_left=sale_policy_lapse, df_right=at_inception_data.df1, how='left')

In [51]:
at_inception_data.df_merged


,policy_id,policy_name,age,benefits_count,education,gender,income,lead_provider_name,lead_type,occupation,...,# calls,# calls contacted,res_code_QA_sum,res_code_claims_sum,res_code_no interest_sum,res_code_other_sum,res_code_payment_sum,res_code_quote_sum,res_code_sale_sum,res_code_update_sum
0,100003c9-2094-3107-e604-65d8e6b4ec03,MWL008050072,51,1.0,Matric,Male,40000.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10000cc8-8f08-5e2c-b6ad-661d48606710,ML24-0199554,23,1.0,Matric + 3 year Diploma,Female,20000.0,Recycled Freemium,External Call Centre,Administrator,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100307d2-6857-7f50-833c-66b391c37879,ML24-0254853,63,1.0,Matric + 3 year Diploma,Female,45000.0,SPL-Freemium,Affiliate,Personal Assistant,...,6.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,0.0
3,10048ce0-253d-aeff-4f59-64e76756bddf,ML23-0067459,41,1.0,Matric + 3 year Diploma,Male,70000.0,Sold Freemium,External Call Centre,Chief Technology Officer (CTO) (< 20% travel),...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10061226-ed6a-b733-47f0-65fb970086cd,MWL007912004,36,5.0,4 year Degree,Female,40000.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65359,ffaa032e-2a01-cf52-cc1a-65fb3147c393,MWL006676786,51,1.0,Matric + 3 year Diploma,Female,40000.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65360,ffb7087c-78f7-a485-5b7f-65d8d25da2c7,MWL007292690,41,1.0,Matric,Female,14000.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65361,ffd20919-303c-e88e-fb1f-65fba1578790,MWL008568948,26,2.0,Matric + 3 year Diploma,Female,14000.0,NaN,NaN,NaN,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
65362,ffe1059c-beea-354d-27fa-6629fe94a5cf,MWL006194665,45,1.0,3 year Degree,Female,30000.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
at_inception_data.df_merged.columns

In [ ]:
#making missing values 0 for specified columns
for col in ['# calls',
      '# calls contacted', 'res_code_QA_sum', 'res_code_claims_sum',
      'res_code_no interest_sum', 'res_code_other_sum',
      'res_code_payment_sum', 'res_code_quote_sum', 'res_code_sale_sum',
      'res_code_update_sum','total_funeral_premium', 'funeral_count']:
    at_inception_data.df_merged[col] = at_inception_data.df_merged[col].apply(lambda x: x if pd.notnull(x) else 0)

In [ ]:
#Replace missing orginal_premium values with premium
at_inception_data.df_merged['original_premium'] = at_inception_data.df_merged.apply(
    lambda row: row['original_premium'] 
    if pd.notnull(row['original_premium'])
    else row['premium'],
    axis=1)

In [ ]:
#addining new column: difference in the original and actual policy start date
at_inception_data.df_merged['duration_policy_start_delay'] =  ((at_inception_data.df_merged[ 'cover_start_date'] - at_inception_data.df_merged['cover_start_date_original']) / np.timedelta64(1, 'D'))//30

C:\Users\E1005279\AppData\Local\Temp\ipykernel_10364\3172697092.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  at_inception_data.df_merged['duration_policy_start_delay'] =  ((at_inception_data.df_merged[ 'cover_start_date'] - at_inception_data.df_merged['cover_start_date_original']) / np.timedelta64(1, 'D'))//30


In [ ]:
#ordering the columns
at_inception_data.df_merged = at_inception_data.df_merged[['policy_id', 'policy_name', 'age', 'education',
       'gender', 'income', 'smoker_status','sold_socio_economic_class','occupation_class','occupation','orginal individual_income','partner_income',
       'lead_provider_name', 'lead_type', 'sales_channel','sale_date', 'campaign_name',
       'underwriting_outcome','optionality', 'hiv_test_required','pricing_version', 'eml', 'pml','hiv_group',
       'securitygroup_id','policy_type','benefits_count','last_benefit_amount','cover_amount_full','total_funeral_premium','funeral_count','original_premium','fixed_debit_day','duration_policy_start_delay',
       '# calls','# calls contacted', 'res_code_QA_sum', 'res_code_claims_sum','res_code_no interest_sum', 'res_code_other_sum',
       'res_code_payment_sum', 'res_code_quote_sum', 'res_code_sale_sum','res_code_update_sum',
       'policy_status','current_policy_status','cancellation_effective_date', 'cancellation_reason', 'lapse_type', 'lapse_flag',  'expected_lapse_rate'
       , 'cover_start_date' ]] # use cover_start_date, cancellation_effective_date to get econimic indexes

In [57]:
at_inception_data.df_merged

,policy_id,policy_name,age,education,gender,income,smoker_status,sold_socio_economic_class,occupation_class,occupation,...,res_code_quote_sum,res_code_sale_sum,res_code_update_sum,policy_status,current_policy_status,cancellation_effective_date,cancellation_reason,lapse_type,lapse_flag,expected_lapse_rate
0,100003c9-2094-3107-e604-65d8e6b4ec03,MWL008050072,51,Matric,Male,40000.0,S,1.0,None,NaN,...,0.0,0.0,0.0,Active,Active,NaT,NaN,NaN,False,0.015060
1,10000cc8-8f08-5e2c-b6ad-661d48606710,ML24-0199554,23,Matric + 3 year Diploma,Female,20000.0,N,2.0,A,Administrator,...,0.0,0.0,0.0,Active,Active,NaT,NaN,NaN,False,0.036555
2,100307d2-6857-7f50-833c-66b391c37879,ML24-0254853,63,Matric + 3 year Diploma,Female,45000.0,E,1.0,A,Personal Assistant,...,0.0,5.0,0.0,Active,Active,NaT,NaN,NaN,False,0.040727
3,10048ce0-253d-aeff-4f59-64e76756bddf,ML23-0067459,41,Matric + 3 year Diploma,Male,70000.0,S,1.0,A,Chief Technology Officer (CTO) (< 20% travel),...,0.0,0.0,0.0,Active,Active,NaT,NaN,NaN,False,0.030750
4,10061226-ed6a-b733-47f0-65fb970086cd,MWL007912004,36,4 year Degree,Female,40000.0,N,1.0,None,NaN,...,0.0,0.0,0.0,Active,Active,NaT,NaN,NaN,False,0.014305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65359,ffaa032e-2a01-cf52-cc1a-65fb3147c393,MWL006676786,51,Matric + 3 year Diploma,Female,40000.0,N,1.0,None,NaN,...,0.0,0.0,0.0,Active,Active,NaT,NaN,NaN,False,0.009601
65360,ffb7087c-78f7-a485-5b7f-65d8d25da2c7,MWL007292690,41,Matric,Female,14000.0,N,2.0,None,NaN,...,0.0,0.0,0.0,Active,Active,NaT,NaN,NaN,False,0.012001
65361,ffd20919-303c-e88e-fb1f-65fba1578790,MWL008568948,26,Matric + 3 year Diploma,Female,14000.0,N,2.0,None,NaN,...,0.0,0.0,0.0,Active,Active,NaT,NaN,NaN,False,0.017760
65362,ffe1059c-beea-354d-27fa-6629fe94a5cf,MWL006194665,45,3 year Degree,Female,30000.0,N,1.0,None,NaN,...,0.0,0.0,0.0,Active,Active,NaT,NaN,NaN,False,0.009202


## post sale (obj2)

In [25]:
post_sale_data = multi_data_ops(file_list=[],df1=post_sale_calls_data, df2=sales_data,df3=policy_data,df4=payment_hist_data, df5=lapse_data)
post_sale_data.col_intersect

{'df1 | df2': {'policy_name'},
 'df1 | df3': {'policy_name'},
 'df1 | df4': set(),
 'df1 | df5': {'policy_name'},
 'df2 | df3': {'campaign_name', 'policy_id', 'policy_name'},
 'df2 | df4': {'policy_id', 'sales_channel'},
 'df2 | df5': {'education', 'gender', 'policy_name', 'smoker_status'},
 'df3 | df4': {'policy_id'},
 'df3 | df5': {'policy_name'},
 'df4 | df5': set()}

In [26]:
post_sale_data.columns

{'df1': Index(['policy_name', '# calls', '# calls contacted', 'res_code_QA_sum',
        'res_code_claims_sum', 'res_code_other_sum', 'res_code_payment_sum',
        'res_code_sale_sum', 'res_code_update_sum'],
       dtype='object'),
 'df2': Index(['policy_id', 'policy_name', 'age', 'benefits_count', 'calls',
        'call_contacts', 'call_contact_rate', 'campaign_name', 'education',
        'gender', 'income', 'lead_provider_name', 'lead_type', 'occupation',
        'payment_profile', 'sales_channel', 'smoker_status',
        'sold_socio_economic_class', 'underwriting_outcome'],
       dtype='object'),
 'df3': Index(['Unnamed: 0.1', 'Unnamed: 0', 'securitygroup_id', 'policy_name',
        'sales_process_name', 'campaign_code', 'campaign_name', 'optionality',
        'hiv_test_required', 'policy_type', 'primary_contact_id',
        'policy_status', 'premium', 'cover_start_date_original',
        'cover_start_date', 'debit_order_due_date',
        'cancellation_effective_date', 'cancel

In [ ]:
sales_policy = post_sale_data.merging(on='policy_name',df_left=post_sale_data.df2, df_right=post_sale_data.df3, how='inner', col_left=['policy_id', 'policy_name', 'age', 'benefits_count', 
        'education','gender', 'income', 'lead_provider_name', 'lead_type', 'occupation',
        'payment_profile', 'sales_channel', 'smoker_status',
        'sold_socio_economic_class', 'underwriting_outcome'], col_right=['securitygroup_id', 'policy_name','campaign_name','optionality',
        'hiv_test_required', 'policy_type', 'policy_status', 'premium', 'cover_start_date_original','cover_start_date', 
        'cancellation_effective_date','sale_date', 'original_premium',
        'cancellation_reason', 'anniversary_due', 'last_benefit_amount',
        'first_month_at_risk', 'last_month_at_risk', 'last_settled', 'SEC',
        'pricing_version', 'first_collected_date',
        'number_of_collection_attempts', 'number_of_successful_collections',
        'total_collected',  'net_collected',
        'last_main_premium', 'last_benefit_type', 'total_funeral_premium',
        'funeral_count', 'has_been_recaptured', 'reason', 'last_benefit_status', 'prev_premium', 'fixed_debit_day',
        'lapse_flag'])

In [28]:
sales_policy.shape

(65364, 49)

In [30]:
sale_policy_lapse = post_sale_data.merging(on='policy_name',df_left=sales_policy, df_right=post_sale_data.df5, how='inner', col_right=['policy_name', 'cover_amount_full', 'current_policy_status',
         'eml', 'pml', 'expected_lapse_rate', 'hiv_group',
        'current individual_income', 'orginal individual_income', 'lapse_type',
        'occupation_class', 'partner_income'])

In [31]:
sale_policy_lapse.shape

(65364, 60)

In [ ]:
sale_policy_lapse_pay = post_sale_data.merging(on='policy_id',df_left=sale_policy_lapse, df_right=post_sale_data.df4, how='inner', col_right=['policy_id', '* policy start delay months',
        '* policy duration months', 'last premium amount due',
        'duration to anniversary', 'payment_method_DebiCheck_sum',
        'payment_method_EFT_sum', 'payment_method_Pre Fund_sum',
        '# anniversaries', 'payment rate', '1 month ago payment',
        '2 month ago payment', '3 month ago payment', '4 month ago payment',
        '# claims'])

In [33]:
sale_policy_lapse_pay.shape

(63981, 74)

In [34]:
post_sale_data.df_merged = post_sale_data.merging(on='policy_name',df_left=sale_policy_lapse_pay, df_right=post_sale_data.df1, how='left')

In [39]:
post_sale_data.df_merged

,policy_id,policy_name,age,benefits_count,education,gender,income,lead_provider_name,lead_type,occupation,...,4 month ago payment,# claims,# calls,# calls contacted,res_code_QA_sum,res_code_claims_sum,res_code_other_sum,res_code_payment_sum,res_code_sale_sum,res_code_update_sum
0,100003c9-2094-3107-e604-65d8e6b4ec03,MWL008050072,51,1.0,Matric,Male,40000.0,NaN,NaN,NaN,...,paid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10000cc8-8f08-5e2c-b6ad-661d48606710,ML24-0199554,23,1.0,Matric + 3 year Diploma,Female,20000.0,Recycled Freemium,External Call Centre,Administrator,...,paid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100307d2-6857-7f50-833c-66b391c37879,ML24-0254853,63,1.0,Matric + 3 year Diploma,Female,45000.0,SPL-Freemium,Affiliate,Personal Assistant,...,paid,0.0,4.0,1.0,0.0,0.0,0.0,3.0,1.0,0.0
3,10048ce0-253d-aeff-4f59-64e76756bddf,ML23-0067459,41,1.0,Matric + 3 year Diploma,Male,70000.0,Sold Freemium,External Call Centre,Chief Technology Officer (CTO) (< 20% travel),...,paid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10061226-ed6a-b733-47f0-65fb970086cd,MWL007912004,36,5.0,4 year Degree,Female,40000.0,NaN,NaN,NaN,...,paid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63976,ffaa032e-2a01-cf52-cc1a-65fb3147c393,MWL006676786,51,1.0,Matric + 3 year Diploma,Female,40000.0,NaN,NaN,NaN,...,paid,0.0,6.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0
63977,ffb7087c-78f7-a485-5b7f-65d8d25da2c7,MWL007292690,41,1.0,Matric,Female,14000.0,NaN,NaN,NaN,...,paid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
63978,ffd20919-303c-e88e-fb1f-65fba1578790,MWL008568948,26,2.0,Matric + 3 year Diploma,Female,14000.0,NaN,NaN,NaN,...,paid,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
63979,ffe1059c-beea-354d-27fa-6629fe94a5cf,MWL006194665,45,1.0,3 year Degree,Female,30000.0,NaN,NaN,NaN,...,paid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
post_sale_data.df_merged.columns

Index(['policy_id', 'policy_name', 'age', 'benefits_count', 'education',
       'gender', 'income', 'lead_provider_name', 'lead_type', 'occupation',
       'payment_profile', 'sales_channel', 'smoker_status',
       'sold_socio_economic_class', 'underwriting_outcome', 'securitygroup_id',
       'optionality', 'hiv_test_required', 'policy_type', 'policy_status',
       'premium', 'cover_start_date_original', 'cover_start_date',
       'cancellation_effective_date', 'sale_date', 'original_premium',
       'cancellation_reason', 'anniversary_due', 'last_benefit_amount',
       'first_month_at_risk', 'last_month_at_risk', 'last_settled', 'SEC',
       'pricing_version', 'first_collected_date',
       'number_of_collection_attempts', 'number_of_successful_collections',
       'total_collected', 'net_collected', 'last_main_premium',
       'last_benefit_type', 'total_funeral_premium', 'funeral_count',
       'has_been_recaptured', 'reason', 'last_benefit_status', 'prev_premium',
       'fixe

In [ ]:
#making missing values 0 for specified columns
for col in ['# calls', '# calls contacted',
       'res_code_QA_sum', 'res_code_claims_sum', 'res_code_other_sum',
       'res_code_payment_sum', 'res_code_sale_sum', 'res_code_update_sum','total_funeral_premium', 'funeral_count']:
    post_sale_data.df_merged[col] = post_sale_data.df_merged[col].apply(lambda x: x if pd.notnull(x) else 0)

In [ ]:
post_sale_data.df_merged = post_sale_data.df_merged[['policy_id', 'policy_name', 'age','education','gender', 'income','current individual_income', 'orginal individual_income', 'partner_income','occupation','occupation_class', 'smoker_status',
 'sold_socio_economic_class',
 'lead_provider_name', 'lead_type','sales_channel', 'campaign_name',
 '# calls', '# calls contacted','res_code_QA_sum', 'res_code_claims_sum', 'res_code_other_sum','res_code_payment_sum', 'res_code_sale_sum', 'res_code_update_sum',
  'securitygroup_id','benefits_count', 'policy_type', 'original_premium','premium','last_main_premium', 'prev_premium','last premium amount due',
  '* policy start delay months', '* policy duration months','duration to anniversary','cover_amount_full','last_benefit_amount','fixed_debit_day', 'total_funeral_premium', 'funeral_count',
'# anniversaries','has_been_recaptured', 'reason', 
'underwriting_outcome','optionality', 'hiv_test_required','pricing_version', 'eml', 'pml','hiv_group',
'number_of_collection_attempts', 'number_of_successful_collections','total_collected', 'net_collected', 'payment_method_DebiCheck_sum', 'payment_method_EFT_sum','payment_method_Pre Fund_sum',
'payment rate','1 month ago payment', '2 month ago payment', '3 month ago payment', '4 month ago payment', '# claims',
'policy_status','last_benefit_status', 'current_policy_status','lapse_type','cancellation_effective_date',  'cancellation_reason',  'lapse_flag', 'expected_lapse_rate'
   ,'cover_start_date']]
      
         
        
       
        
      
          

# Merging with functions

In [5]:
inception_data = data_tr.at_inception_data_merge(pre_inception_calls_data, sales_data,policy_data,lapse_data)
near_ftr_lapse_data = data_tr.post_sale_data_merge(post_sale_calls_data, sales_data,policy_data,lapse_data,payment_hist_data)

In [7]:
inception_data.to_csv(path+'inception_data.csv')
near_ftr_lapse_data.to_csv(path+'near_ftr_lapse_data.csv')